# Import bibliotek

In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, activations
from tensorflow.python.keras.callbacks import ModelCheckpoint
import torch

In [2]:
gpus = tf.config.list_physical_devices()
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:CPU:0   Type: CPU
Name: /physical_device:GPU:0   Type: GPU


# Zmienne

In [3]:
SPECTROGRAM = False
classes = ["Anger", "Happy", "Neutral", "Sad", "Fear", "Disgust"]

In [4]:
if SPECTROGRAM:
    path_train = "../data/spectrograms/divided/Crema/train"
    path_test = "../data/spectrograms/divided/Crema/test"
    log_directory = "../tests/logs/TensorBoard/spec_crema"
    filepath = "../tests/logs/SavedModels/spec_crema.h5"

else:
    path_train = "../data/melspectrograms/divided/Crema/train"
    path_test = "../data/melspectrograms/divided/Crema/test"
    log_directory = "../tests/logs/TensorBoard/melspec_crema"
    filepath = "../tests/logs/SavedModels/melspec_crema.h5"

In [5]:
def get_train_data(train_data_path):
    """
    Loads train data from all datasets

    Returns:
        data_train - training samples
        data_val - validation samples
        target_train - training targets
        target_val - validation targets
        
    """
    class_number = 0
    targets = []
    img = []

    for current_folder in classes:
        print(f"Emotion: {current_folder}")
        emotion_folder = f"{train_data_path}/{current_folder}"
        for i, file_name in enumerate(os.listdir(emotion_folder)):
            file_path = f"{emotion_folder}/{file_name}"
            targets.append(class_number)
            img.append(np.array(Image.open(file_path).convert('RGB'))/255)
            print(f"Loaded {i+1}/{len(os.listdir(emotion_folder))} files from train {current_folder}", end="\r")
        print("\n")
        class_number += 1

    
    targets_array = np.asarray(targets)
    print("Set np.asarray(targets)")
    targets = []
    img_array = np.asarray(img)
    print("Set np.asarray(img)")
    img = []

    targets_array = tf.keras.utils.to_categorical(targets_array)
    print("Set tf.keras.utils.to_categorical(targets_array)")
    
    data_train, data_val, target_train, target_val = train_test_split(img_array, targets_array, test_size=0.25, random_state=0)
    print("Splitted to train/val datasets")
    
    print("shapes")
    print(data_train.shape)
    print(target_train.shape)
    print(data_val.shape)
    print(target_val.shape)

    return data_train, data_val, target_train, target_val

In [6]:
def get_test_data(test_data_path):
    """
    Loads test data from all datasets

    Returns:
        data_test - test samples
        target_test_to_categorical - test targets
        
    """
    class_number = 0
    targets = []
    img = []

    for current_folder in classes:
        print(f"Emotion: {current_folder}")
        emotion_folder = f"{test_data_path}/{current_folder}"
        for i, file_name in enumerate(os.listdir(emotion_folder)):
            file_path = f"{emotion_folder}/{file_name}"
            targets.append(class_number)
            img.append(np.array(Image.open(file_path).convert('RGB'))/255)
            print(f"Loaded {i+1}/{len(os.listdir(emotion_folder))} files from test {current_folder}", end="\r")
        print("\n")
        class_number += 1

    target_test = np.asarray(targets)
    print("Set np.asarray(targets)")
    targets = []
    data_test = np.asarray(img)
    print("Set np.asarray(img)")
    img = []
    
    target_test_to_categorical = tf.keras.utils.to_categorical(target_test)
    print("Set tf.keras.utils.to_categorical(target_test)")

    print(data_test.shape)
    print(target_test_to_categorical.shape)

    return data_test, target_test_to_categorical

In [7]:
# MODEL SHEDULER
def scheduler(epoch, lr):
    if epoch < 15:
        eta = lr
    else:
        eta = lr - 0.00001
    if lr < 0.00005:
        eta = 0.00005

    return eta

In [8]:
data_train, data_val, target_train, target_val = get_train_data(path_train)

Emotion: Anger
Loaded 1017/1017 files from train Anger

Emotion: Happy
Loaded 1016/1016 files from train Happy

Emotion: Neutral
Loaded 869/869 files from train Neutral

Emotion: Sad
Loaded 1017/1017 files from train Sad

Emotion: Fear
Loaded 1017/1017 files from train Fear

Emotion: Disgust
Loaded 1017/1017 files from train Disgust

Set np.asarray(targets)
Set np.asarray(img)
Set tf.keras.utils.to_categorical(targets_array)
Splitted to train/val datasets
shapes
(4464, 389, 515, 3)
(4464, 6)
(1489, 389, 515, 3)
(1489, 6)


In [9]:
# MODEL SHEDULER
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# TENSORBOARD
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_directory)

# SAVE MODEL 
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_loss', 
                             verbose=1,
                             save_best_only=True,
                             mode='min')


model = models.Sequential()
model.add(layers.Conv2D(75, (5, 5), activation='relu', input_shape=(389, 515, 3), padding='same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(135, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Dropout(0.15))
model.add(layers.Conv2D(75, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten()) 
model.add(layers.Dense(45, activation='relu'))
model.add(layers.Dropout(0.2))
# model.add(layers.Dense(40, activation='relu'))
# model.add(layers.Dropout(0.3))
model.add(layers.Dense(6, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 389, 515, 75)      5700      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 129, 171, 75)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 129, 171, 135)     253260    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 57, 135)      0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 43, 57, 135)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 43, 57, 75)        2

In [10]:
# MODEL COMPILE 
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [11]:
# MODEL FIT 
history = model.fit(data_train,
                    target_train,
                    epochs=20,
                    shuffle=True,
                    validation_data=(data_val, target_val),
                    callbacks=[tensorboard_callback, checkpoint])

Epoch 1/20


2024-10-29 16:01:38.935632: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


140/140 [==============================] - ETA: 0s - loss: 1.7229 - accuracy: 0.2650
Epoch 00001: val_loss improved from inf to 1.62569, saving model to ../tests/logs/SavedModels/melspec_crema.h5
140/140 [==============================] - 478s 3s/step - loss: 1.7229 - accuracy: 0.2650 - val_loss: 1.6257 - val_accuracy: 0.3479
Epoch 2/20
140/140 [==============================] - ETA: 0s - loss: 1.6117 - accuracy: 0.3230
Epoch 00002: val_loss improved from 1.62569 to 1.57657, saving model to ../tests/logs/SavedModels/melspec_crema.h5
140/140 [==============================] - 285s 2s/step - loss: 1.6117 - accuracy: 0.3230 - val_loss: 1.5766 - val_accuracy: 0.3559
Epoch 3/20
140/140 [==============================] - ETA: 0s - loss: 1.6343 - accuracy: 0.3461
Epoch 00003: val_loss improved from 1.57657 to 1.56889, saving model to ../tests/logs/SavedModels/melspec_crema.h5
140/140 [==============================] - 282s 2s/step - loss: 1.6343 - accuracy: 0.3461 - val_loss: 1.5689 - val_acc

In [12]:
data_train = []
data_val = []
target_train = []
target_val = []

targets = []
img = []

data_test, target_test_to_categorical = get_test_data(path_test)

Emotion: Anger
Loaded 254/254 files from test Anger

Emotion: Happy
Loaded 255/255 files from test Happy

Emotion: Neutral
Loaded 218/218 files from test Neutral

Emotion: Sad
Loaded 254/254 files from test Sad

Emotion: Fear
Loaded 254/254 files from test Fear

Emotion: Disgust
Loaded 254/254 files from test Disgust

Set np.asarray(targets)
Set np.asarray(img)
Set tf.keras.utils.to_categorical(target_test)
(1489, 389, 515, 3)
(1489, 6)


In [13]:
model.load_weights(filepath=filepath)
    
results = model.evaluate(data_test, target_test_to_categorical, batch_size=1)
print("test loss, test acc:", results)

1489/1489 [==============================] - 39s 26ms/step - loss: 1.4211 - accuracy: 0.4567
test loss, test acc: [1.4211245775222778, 0.45668232440948486]
